# SIRF reconstruction using data simulated with SIMIND ###
This notebook lays out a simple simulation and reconstruction using SIMIND and SIRF \
Please see Rebecca Gillen's instructions for a more in depth guide

### simind can be donwloaded from https://simind.blogg.lu.se/downloads/
There are reasonably straight forward instructions to be followed for Windows/Mac/Linux
### SIRF can be downloaded from https://github.com/SyneRBI/SIRF
There are more complicated, but logical instructions for building the required packaged and libraries at https://github.com/SyneRBI/SIRF/wiki/Installation-instructions

In [ ]:
%%bash
pip install pandas
pip install numpy

In [ ]:
import sys
sys.path.append("//home/sam/devel/STIR/STIR_github/STIR/src/swig") # not sure why VScode doesn't recongnise this path without me doing this

In [ ]:
import stir
import stirextra

import numpy as np
import pandas as pd
import pylab

Below we define two functions (the first of which should hopefully become necessary) in order to set up a voxel-based phantom (to use SIMIND's language)

In [ ]:
def create_sample_image(image):
    '''fill the image with some geometric shapes to create a simple phantom image'''
    im_shape = image.shape()
    tmp_im = image.get_empty_copy()
    
    shapeim = tmp_im.get_empty_copy()
    
    # create a body-like ellipsoid shape
    shape = stir.EllipsoidalCylinder()
    shape.set_length(400)
    shape.set_radius_x(im_shape[2]//2*3)
    shape.set_radius_y(im_shape[1]//3*4)
    shape.scale = (150)
    shape.set_origin((stir.FloatCartesianCoordinate3D(0,0,0)))
    shape.construct_volume(shapeim, stir.IntCartesianCoordinate3D(1,1,1))
    tmp_im.xapyb(tmp_im,1,shapeim,1)

    # add some lung-like shapes shape
    shape.set_radius_x(im_shape[2]//2)
    shape.set_radius_y(im_shape[1])
    shape.set_origin((stir.FloatCartesianCoordinate3D(0,0,-im_shape[2]//3*2)))
    shape.construct_volume(shapeim, stir.IntCartesianCoordinate3D(1,1,1))
    tmp_im.xapyb(tmp_im,1,shapeim,-0.5)

    shape.set_origin((stir.FloatCartesianCoordinate3D(0,0, im_shape[2]//3*2)))
    shape.construct_volume(shapeim, stir.IntCartesianCoordinate3D(1,1,1))
    tmp_im.xapyb(tmp_im,1,shapeim,-0.5)

    # a spine-like shape
    shape.set_radius_x(im_shape[2]//4)
    shape.set_radius_y(im_shape[1]//4)
    shape.set_origin((stir.FloatCartesianCoordinate3D(0, im_shape[1]//5*4, 0)))
    shape.construct_volume(shapeim, stir.IntCartesianCoordinate3D(1,1,1))
    tmp_im.xapyb(tmp_im,1,shapeim,0.5)

    return tmp_im

We are now ready to create our image. We require only a template sinogram containing information such as the image dimensions and number of projections (among many other things - take a look at the .hs file)

In [ ]:
templ_sino = stir.ProjData.read_from_file("./template_sinogram.hs") # template sirf acquisition data


In [ ]:
templ_sino.get_max_tangential_pos_num()

In [ ]:
templ_sino.get_max_view_num()

In [ ]:
image = stir.FloatVoxelsOnCartesianGrid(templ_sino.get_proj_data_info(), 1)
im = stir.zoom_image(create_sample_image(image), 1, 0, 0, 64) # use the template to create an empty image
pylab.figure()
pylab.subplot(1, 2, 1)
pylab.imshow(np.squeeze(stirextra.to_numpy(im)))
pylab.title('Empty Image')
pylab.show(block=False)

In [ ]:
att_STIR = im.get_empty_copy()
att_STIR.fill((stirextra.to_numpy(im)/1000).flat) # approximate attenuation coefficient image used in STIR (/cm)
att_SIMIND = im.get_empty_copy()
att_SIMIND.fill((stirextra.to_numpy(im)/0.15).flat) # approximate density image used in SIMIND (mg/cm^3)
att_none = im.get_empty_copy() # zero density map to investigate the effects on SIMIND reconstruction

And we have a simple image phantom

We now write these images to file.
SIMIND requires a .dmi file for an attenuation (density) image and a .smi for an emission (source) image\
We can easily do this using the ImageData.write() method

In [ ]:
im.write_to_file("./image.smi")
att_STIR.write_to_file("./attenuation_stir.hv")
att_SIMIND.write_to_file("./attenuation_simind.dmi")
att_none.write_to_file("./attenuation_none.dmi")

Now for a bit of fun, we'll use SIMIND's tumour functionality. 
We need to create a space separated value file where
* The first three values (1-3) define the centre of the tumour in pixel units.
* The next values (4-6) refer to the centre location of the tumour also in pixel units.
* Value 7 is the voxel value and is a relative value associated with the main source
distribution.
* Value 8 is the density of the tumour. 
* Value 9 and 10 determine the distribution of activity in the tumour. If both values are 0,
then it will be a uniform distributed activity.

We will define a uniform tumour (1, 5, 5) pixels wide located at (0, 20 , 30) using a panda data frame

In [ ]:
tumour_arr = np.array([[1.,5.,5.,0,20.,30.,10.,5000.,0., 0.], [10.,15.,15.,0, 50.,45.,1000.,5000.,0., 0.]])
tumour_df = pd.DataFrame(tumour_arr)
tumour_df.to_csv('./tumour.inp', index=False, header=False, sep=' ')

We're now ready to simulate our emission data. We have a .smc file containing information about the simulation. Please read the simind manual to learn about the many different options available. These options can be altered using either the change command (type "change input.smc into the terminal) or using switches \
The syntax for reconstruction is as follows:
`simind input_file_prefix outpute_files_prefix`
This can be followed by switches seperated by forward slashes such as below \*\
\* Unfortunately this causes some trible with Linux & MacOS file directories. The SIMIND manual claims that two backslashed '\\' can be used in place of a forward slash that is part of a file directory, but I haven't found this to be the case

The following bash command defines a .smc file `input.smc` follwed by a prefix for output files `output` \
Switches are then used to define:
* /NN: a multiplier for the number of histories per projection (which is calculated using the sum of all voxel values)
* /PX: defines the image pixel size in the i,j direction (transverse in this case) - im.voxel_sizes()
* /FS: defines the prefix for the .smi emission image file
* /FD: defines the prefix for rhe .dmi attenuation image file
* 

In [ ]:
%%bash
simind input output/NN:0.001/PX:0.4/FS:image.smi/FD:attenuation_simind.dmi/IF:tumour.inp

And (assuming the preious cell ran) we have now simulated our SPECT data!\
Next we need to get this data into a format the SIRF will recognise. Luckily we have a script ready that does this for us.
This script changes a few lines in the data's header file and the header file suffix. Differences between the conventions of interfiles in SIMIND and STIR/SIRF can be found in Rebecca's notes.

In [ ]:
%%bash
sh ./convertSIMINDToSTIR.sh output.h00

We can now view the resulting sinogram

In [ ]:
simulated_data =  stir.ProjData.read_from_file("./output.hs")
pylab.figure()
pylab.subplot(1, 2, 1)
pylab.imshow(np.squeeze(stirextra.to_numpy(simulated_data)))
pylab.title('Empty Image')
pylab.show(block=False)

OK, so now we have our projection data in a format that SIRF likes, we can go about reconstructing the data. In this notebook we do this as simply as possible with a back prokection. \
In order to do this we first need to create our acqusition model matrix

In [ ]:
help(simulated_data)

In [ ]:
simulated_data.fill(np.maximum(stirextra.to_numpy(simulated_data), 0.0001).flat) # make positive - shouldn't need to do this!

# Nothing Works from here on in
## Need to get SPECTUB Matrix usable with Python

In [ ]:
acq_model_matrix = stir.ProjMatrixByBinSPECTUB()
#acq_model_matrix.set_attenuation_image(att_STIR) # set the attenuation image for reconstruction 
acq_model_matrix.set_resolution_model(0.1,0.1,full_3D=False) # where we have defnied our collimator blurring as a gaussian with SD 0.1mm and a collimator slope of 0.1mm

We then use this matrix to set up an acqusition model. This requires the set-Up() method to be run and needs a projection and image template 

In [ ]:
am = stir.
am.set_up(data, im)

And we can finally backproject the data

In [ ]:
res = am.backward(data)
res.show()

Below is a forward and backproject of the data using SIRF only for comparison (obvisouly this doesn't model noise in any way)

In [ ]:
forward_projected_data = am.forward(im)
forward_projected_data.show()
forward_projected_data.dimensions()

In [ ]:
back_projected_image = am.backward(forward_projected_data)
back_projected_image.show()

In [ ]:
obj_function = STIR.make_Poisson_loglikelihood(forward_projected_data)
obj_function.set_acquisition_model(am)

reconstructor = STIR.OSMAPOSLReconstructor()
reconstructor.set_objective_function(obj_function)
reconstructor.set_num_subsets(1)
reconstructor.set_num_subiterations(20)

init = im.get_uniform_copy(1)
filter = STIR.TruncateToCylinderProcessor()
filter.apply(init)
init.show()

reconstructor.set_up(init)

In [ ]:
reconstructor.reconstruct(init)

In [ ]:
out = reconstructor.get_current_estimate()
out.show()

This last cell removes any temporary image files that SIRF occassionally misses

In [ ]:
import glob
import os

for f in glob.glob(("./tmp*")):
    os.remove(f)